# 미션9

FashionMNIST 데이터셋의 각 패션 아이템(예: 티셔츠, 바지, 스니커즈 등)을 조건부로 생성하는 작업을 수행합니다.

각 클래스에 해당하는 이미지를 생성하는 cGAN (Conditional GAN) 모델을 직접 설계하고 학습시켜 보세요.

**데이터 구성**:
- **훈련 데이터**: 60,000장의 이미지
- **테스트 데이터**: 10,000장의 이미지
- 28×28 크기의 흑백 이미지 **(10개 클래스)**

**클래스 목록**:
- T-shirt/top
- Trouser
- Pullover
- Dress
- Coat
- Sandal
- Shirt
- Sneaker
- Bag
- Ankle boot

### 전체 흐름 요약

1. 데이터 준비 – FashoinMNIST 데이터셋을 다운 받아 정규화(-1~1) 시키고 -> 라벨 One-Hot 또는 임베딩을 준비합니다.
2. DataLoader 정의 - 배치(batch)size, 셔플, 멀티 스레드 로딩합니다.
3. 모델 설계 – Generator - 조건부 입력 (노이즈+라벨) -> `convTranspose2d` 블록 + Conditional BatchNorm
4. 모델 설계 – Discriminator - 조건부 입력(이미지 + 라벨) -> `Conv2d` 블록 + Projection Discriminator
5. 손실·옵티마이저 정의 - BCELoss 또는 WGAN‑GP → Adam/AdamW
6. 학습 루프 구현 - 에포크, 디스크리미네이터·제너레이터 순차 업데이트, 로깅, 체크포인트 저장
7. 중간·최종 결과 시각화 - 클래스별 샘플 이미지 그리드, 학습 로스·시각화
8. 정량적 평가 - FID(Fréchet Inception Distance), IS(Inception Score) 혹은 K‑NN 기반 평가
9. 모델 저장·로드 & 인퍼런스 함수 - .pth 저장·로드, generate(class_id, n_samples)

아래에서 각 단계별 핵심 코드를 한 줄씩 짚어 보겠습니다.

### 프로젝트 할 일 목록 (ToDO List)

1. 환경 설정 및 데이터 로드: 필요한 라이브러리를 임포트하고 Fashion-MNIST 데이터셋을 불러옵니다.
2. 데이터 전처리 및 시각화: 데이터를 정규화하고, 샘플 이미지를 확인하여 데이터셋의 구성을 이해합니다.
3. cGAN 모델 설계:
   * 생성자 (Generator): 노이즈 벡터와 조건부 레이블을 입력받아 이미지를 생성하는 모델을 설계합니다.
   * 판별자 (Discriminator): 이미지와 조건부 레이블을 입력받아 해당 이미지가 진짜인지 가짜인지 판별하는 모델을
         설계합니다.
4. 학습 환경 설정: 모델, 손실 함수, 옵티마이저를 초기화하고 학습에 필요한 하이퍼파라미터를 정의합니다.
5. cGAN 모델 학습: 생성자와 판별자를 번갈아 학습시키는 훈련 루프를 구현하고, 주기적으로 생성된 이미지를 저장하여 학습
      과정을 모니터링합니다.
6. 조건부 이미지 생성 및 평가: 학습된 모델을 사용하여 각 클래스별 이미지를 생성하고, 생성된 이미지의 품질을 시각적으로 평가합니다.

## 환경 설정 및 데이터 로드

- Python 3.10.19+
- PyTorch 2.5.1+ (CUDA 12 권장)
- torchvision, tqdm, matplotlib, numpy
- 정량적 평가용: torch-fidelity (FID/IS), scikit-learn (K‑NN)

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import pandas as pd

In [26]:
# device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [14]:
# ============================================
# Dataset 및 DataLoader 설정
# ============================================
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # [-1, 1] 범위로 정규화
])

train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# 배치 형태 확인
imgs, lbls = next(iter(train_loader))
print(f'imgs.shape: {imgs.shape}, lbls.shape: {lbls.shape}')
# -> torch.Size([128, 1, 28, 28]) torch.Size([128])

# 클래스 이름 (FashionMNIST 클래스)
idx_to_class = {i: class_name for i, class_name in enumerate(train_dataset.classes)}
print(f'idx_to_class : {idx_to_class}')

imgs.shape: torch.Size([64, 1, 28, 28]), lbls.shape: torch.Size([64])
idx_to_class : {0: 'T-shirt/top', 1: 'Trouser', 2: 'Pullover', 3: 'Dress', 4: 'Coat', 5: 'Sandal', 6: 'Shirt', 7: 'Sneaker', 8: 'Bag', 9: 'Ankle boot'}


## cGAN 모델 정의

조건부 GAN에서 **Generator**는 두 가지 입력을 받습니다.

1. **노이즈 벡터** `z ∈ ℝ^{latent_dim}` (예: 100 차원, 표준 정규분포)
2. **클래스 라벨** `y ∈ {0,…,9}` → 임베딩 → `y_emb ∈ ℝ^{embed_dim}`

두 벡터를 **concatenation**하거나 **Conditional BatchNorm**을 통해 결합합니다.

여기서는 **Concatenation + Linear** 방식과 **Conditional BatchNorm**(optional) 두 가지 구현을 보여줍니다.

In [ ]:
# ============================================
# 생성자 (Generator) 클래스
# ============================================
# class Generator(nn.Module):
#     def __init__(self, latent_dim=100, embed_dim=50, img_shape=(1, 28, 28)):
#         super(Generator, self).__init__()
#         self.img_shape = img_shape
#         self.latent_dim = latent_dim
#         self.embed_dim = embed_dim

#         # 라벨 임베딩 (10 → embed_dim)
#         self.label_emb = nn.Embedding(10, embed_dim)

#         # 입력 차원: latent_dim + embed_dim
#         self.init_size = 7   # 7x7 feature map (28 / 4)
#         self.fc = nn.Sequential(
#             nn.Linear(latent_dim + embed_dim, 128 * self.init_size * self.init_size),
#             nn.BatchNorm1d(128 * self.init_size * self.init_size),
#             nn.LeakyReLU(0.2, inplace=True)
#         )

#         # Upsampling block
#         self.deconv = nn.Sequential(
#             # 128 x 7 x 7 -> 64 x 14 x 14
#             nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
#             nn.BatchNorm2d(64),
#             nn.LeakyReLU(0.2, inplace=True),

#             # 64 x 14 x 14 -> 1 x 28 x 28
#             nn.ConvTranspose2d(64, img_shape[0], kernel_size=4, stride=2, padding=1),
#             nn.Tanh()   # output in [-1, 1]
#         )

#     def forward(self, noise, labels):
#         # embedding
#         label_embedding = self.label_emb(labels)

#         # concat noise + label
#         gen_input = torch.cat((noise, label_embedding), dim=1)   # (batch, latent+embed)

#         out = self.fc(gen_input)
#         out = out.view(out.size(0), 128, self.init_size, self.init_size)   # (B,128,7,7)
#         img = self.deconv(out)
#         return img

In [ ]:
# ============================================
# 생성자 (Generator) 클래스
# ============================================
class Generator(nn.Module):
    def __init__(self, z_dim, num_classes, img_size, channels):
        super(Generator, self).__init__()
        self.img_size = img_size
        self.channels = channels
        
        # 레이블을 위한 임베딩 레이어
        self.label_embedding = nn.Embedding(num_classes, num_classes)
        
        # 노이즈와 임베딩된 레이블을 합친 벡터를 처리하는 모델
        self.model = nn.Sequential(
            # 입력 크기 : z_dim + num_classes
            # 초기 이미지를 만들기 위해 7x7 크기의 256개의 채널로 변환
            nn.Linear(z_dim + num_classes , 256 * 7* 7),
            nn.ReLU(),
            nn.Unflatten(1,(256,7,7)), # (배치, 256, 7, 7),
            nn.ReLU(),
            nn.Unflatten(1,(256,7,7)), # (배치, 256, 7, 7)
            
            # 14x14 로 업샘플링
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False), #(배치, 128, 14, 14)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            # 28x28 업샘플링
            nn.ConvTranspose2d(128,channels, kernel_size=4, stride=2, padding=1, bias=False), #(배치, channels, 28, 28)
            
            # 출력 범위를 -1 ~ 1 정규화
            nn.Tanh()
        )
        
    def forward(self, z, labels):
        # 레이블 임베딩
        c = self.label_embedding(labels)
        # 노이즈와 레이블 결합
        x = torch.cat([z,c],1)
        # 이미지 생성
        output = self.model(x)
        return output

onditional BatchNorm

- **조건부 배치 정규화(CBND)**: 라벨별 `γ, β` 파라미터를 학습하고, `nn.BatchNorm2d` 대신 `ConditionalBatchNorm2d` 구현.
- 이 방법은 **Projection GAN**이나 **Self‑Attention GAN**에 자주 쓰이며, 이미지 품질을 약간 향상시킵니다.

> **참고**: 여기서는 기본 Concatenation 방식만 구현하되, `ConditionalBatchNorm2d` 클래스를 별도 코드 블록에 넣어 `if use_cbn:` 형태로 스위치할 수 있게 합니다.

Discriminator도 라벨 정보를 **조건부**으로 활용합니다.

**두 가지 방식** 중 하나를 선택:

1. **Concat 방식**: 이미지와 라벨 임베딩을 채널 차원에 붙여서 `Conv2d` 로 학습.  
2. **Projection 방식** : 라벨 임베딩을 **feature space**에 내적해 스칼라를 더함 → 논문 *"Conditional Image Synthesis With Auxiliary Classifier GANs"* 에서 영감을 받음.

> **Tip** – `torch.nn.utils.spectral_norm` 를 `nn.Conv2d` 혹은 `nn.Linear` 에 적용하면 **스펙트럴 정규화**가 자동으로 적용돼 훈련 안정성이 크게 향상됩니다. 필요 시 `torch.nn.utils.spectral_norm` 로 래핑해 주세요.

In [16]:
class Discriminator(nn.Module):
    """
    Projection Discriminator (Miyato et al., 2018)
    f(x) = h(x) + <ψ(y), φ(x)>
    where:
        - h(x): scalar output from CNN on image
        - ψ(y): label embedding (size = embed_dim)
        - φ(x): image feature vector (size = embed_dim)
    """
    def __init__(self, embed_dim=50, img_shape=(1, 28, 28)):
        super(Discriminator, self).__init__()
        self.embed_dim = embed_dim
        self.label_emb = nn.Embedding(10, embed_dim)

        # CNN backbone (downsample 28->14->7)
        self.conv = nn.Sequential(
            nn.Conv2d(img_shape[0], 64, kernel_size=4, stride=2, padding=1),   # 1x28x28 -> 64x14x14
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),            # 64x14x14 -> 128x7x7
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True)
        )

        # Final feature -> scalar (h(x))
        self.adv_head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 7 * 7, 1)   # scalar real/fake logit
        )

        # Feature projection to embed_dim (φ(x))
        self.proj = nn.Linear(128 * 7 * 7, embed_dim, bias=False)

    def forward(self, img, labels):
        # img : (B,1,28,28), labels : (B,)
        feats = self.conv(img)                 # (B,128,7,7)
        feats_flat = feats.view(feats.size(0), -1)   # (B,128*7*7)

        # scalar logit from image alone
        out_adv = self.adv_head(feats_flat)    # (B,1)

        # projection term <ψ(y), φ(x)>
        phi = self.proj(feats_flat)            # (B, embed_dim)
        psi = self.label_emb(labels)           # (B, embed_dim)

        proj = torch.sum(phi * psi, dim=1, keepdim=True)   # (B,1)

        # Final discriminator logit
        out = out_adv + proj
        return out.squeeze()   # (B,)

## 손실·옵티마이저 정의

In [17]:
# ============================================
# 전통적인 BCE‑GAN
# ============================================
adversarial_loss = nn.BCEWithLogitsLoss()   # logits 입력 (torch.nn.functional.binary_cross_entropy_with_logits)

# 옵티마이저
lr = 2e-4
b1, b2 = 0.5, 0.999   # Adam 파라미터
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

NameError: name 'generator' is not defined

> `BCELoss` 로 시작하고, 학습이 잘 안 될 경우 `WGAN‑GP` 로 전환을 권장합니다.

In [ ]:
# ============================================
# WGAN‑GP 손실: D(real) - D(fake) + λ * GP
# ============================================
lambda_gp = 10.0

def gradient_penalty(D, real, fake, labels):
    # Random weight for interpolation between real and fake
    alpha = torch.rand(real.size(0), 1, 1, 1, device=real.device)
    interpolates = (alpha * real + ((1 - alpha) * fake)).requires_grad_(True)
    d_interpolates = D(interpolates, labels)
    fake = torch.ones(d_interpolates.size(), device=real.device)
    # Get gradient w.r.t. interpolates
    gradients = torch.autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True)[0]
    gradients = gradients.view(gradients.size(0), -1)
    gp = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gp

## 환경 설정 및 데이터 로드

> **Tip** – `log_interval` 를 `len(train_loader)//10` 정도로 잡으면 매 에포크마다 10번 정도 로스가 출력돼 학습 진행 상황을 빠르게 파악할 수 있습니다.

In [ ]:
# ============================================
# BCE‑GAN
# ============================================

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# 고정된 노이즈와 라벨(시각화용)
fixed_noise = torch.randn(10, 100, device=device)   # 10개 클래스당 1개씩
fixed_labels = torch.arange(10, device=device)      # 0~9

def sample_and_save(epoch):
    generator.eval()
    with torch.no_grad():
        gen_imgs = generator(fixed_noise, fixed_labels).cpu()
        # denorm
        gen_imgs = gen_imgs * 0.5 + 0.5
        grid = torchvision.utils.make_grid(gen_imgs, nrow=5, normalize=False)
        # 저장
        out_path = f'output/epoch_{epoch:04d}.png'
        torchvision.utils.save_image(grid, out_path)
    generator.train()

num_epochs = 100
log_interval = 200   # step마다 로스 출력

for epoch in range(1, num_epochs + 1):
    for i, (imgs, labels) in enumerate(tqdm(train_loader, desc=f'Epoch {epoch}/{num_epochs}')):
        batch_size = imgs.size(0)
        real = imgs.to(device)
        labels = labels.to(device)

        # -----------------
        #  Train Discriminator
        # -----------------
        optimizer_D.zero_grad()

        # Real loss
        real_validity = discriminator(real, labels)
        real_loss = F.binary_cross_entropy_with_logits(
            real_validity, torch.ones_like(real_validity))

        # Fake loss
        z = torch.randn(batch_size, 100, device=device)
        fake_imgs = generator(z, labels).detach()
        fake_validity = discriminator(fake_imgs, labels)
        fake_loss = F.binary_cross_entropy_with_logits(
            fake_validity, torch.zeros_like(fake_validity))

        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()

        # -----------------
        #  Train Generator
        # -----------------
        optimizer_G.zero_grad()

        # Sample new noise
        z = torch.randn(batch_size, 100, device=device)
        gen_imgs = generator(z, labels)
        # Generator tries to fool the discriminator
        validity = discriminator(gen_imgs, labels)
        g_loss = F.binary_cross_entropy_with_logits(
            validity, torch.ones_like(validity))
        g_loss.backward()
        optimizer_G.step()

        # ----- logging -----
        if i % log_interval == 0:
            tqdm.write(f"[E{epoch}/{num_epochs} I{i}/{len(train_loader)}] "
                       f"D loss: {d_loss.item():.4f} | G loss: {g_loss.item():.4f}")

    # ----- epoch-end tasks -----
    # 1) 샘플 이미지 저장
    sample_and_save(epoch)

    # 2) 체크포인트 저장
    torch.save({
        'epoch': epoch,
        'generator_state_dict': generator.state_dict(),
        'discriminator_state_dict': discriminator.state_dict(),
        'optimizer_G_state_dict': optimizer_G.state_dict(),
        'optimizer_D_state_dict': optimizer_D.state_dict(),
    }, f'checkpoints/ckpt_epoch_{epoch:04d}.pth')

WGAN‑GP 차이점
- **Discriminator**를 `n_critic` (예: 5) 번 업데이트
- **Loss**: `d_loss = -(real_validity.mean() - fake_validity.mean()) + λ * gradient_penalty`
- **Generator** loss: `-fake_validity.mean()`

코드 블록을 별도 셀에 넣고, `use_wgan_gp=True` 로 스위치하면 됩니다.

In [ ]:
# ============================================
# WGAN‑GP
# ============================================

### 중간·최종 결과 시각화

In [ ]:
# ============================================
# 샘플 시각화
# ============================================

def show_samples(dataset, n=10):
    fig, axes = plt.subplots(1, n, figsize=(n*1.2, 1.5))
    for i in range(n):
        img, label = dataset[i]
        img = img.squeeze().numpy() * 0.5 + 0.5   # denorm for display
        axes[i].imshow(img, cmap='gray')
        axes[i].set_title(idx_to_class[label], fontsize=8)
        axes[i].axis('off')
    plt.show()

show_samples(train_set, n=10)

In [ ]:
# ============================================
# 클래스별 이미지 그리드
# ============================================
def visualize_classes(generator, n_per_class=8):
    generator.eval()
    all_imgs = []
    all_labels = []
    with torch.no_grad():
        for class_id in range(10):
            # 동일 라벨에 대해 여러 노이즈 샘플링
            z = torch.randn(n_per_class, 100, device=device)
            labels = torch.full((n_per_class,), class_id, dtype=torch.long, device=device)
            gen_imgs = generator(z, labels).cpu()
            all_imgs.append(gen_imgs)
            all_labels.extend([class_names[class_id]] * n_per_class)

    # (10, n_per_class, 1, 28, 28) -> (10*n_per_class, 1, 28, 28)
    grid_imgs = torch.cat(all_imgs, dim=0)
    grid = torchvision.utils.make_grid(grid_imgs, nrow=n_per_class, normalize=True, pad_value=1)

    plt.figure(figsize=(n_per_class, 10))
    plt.title("Conditional Generation (rows: class, cols: samples)")
    plt.imshow(grid.permute(1, 2, 0).numpy())
    plt.axis('off')
    plt.show()
    generator.train()

visualize_classes(generator, n_per_class=8)

> **Tip** – `TensorBoard` 를 사용하려면 `torch.utils.tensorboard.SummaryWriter` 로 `add_scalar('Loss/D', d_loss, global_step)` 등을 기록하면, Colab에서도 `tensorboard --logdir=runs` 로 실시간 모니터링 가능합니다.

In [ ]:
# ============================================
# 로스 곡선 시각화
# ============================================

log_df = pd.read_csv('training_log.csv')   # epoch, step, d_loss, g_loss (CSV 저장해두었다면)

plt.figure(figsize=(8,4))
plt.plot(log_df['step'], log_df['d_loss'], label='Discriminator')
plt.plot(log_df['step'], log_df['g_loss'], label='Generator')
plt.xlabel('Training Step')
plt.ylabel('Loss')
plt.title('Training Curve')
plt.legend()
plt.show()

### 정량적 평가

**Fashion‑MNIST** 은 흑백 28×28 라는 특성 때문에 일반적인 `Inception Score`(IS) 를 쓰기엔 적합도가 낮습니다. 대신 아래 두 지표를 활용할 수 있습니다.

| 지표 | 설명 | 구현 방법 |
|------|------|-----------|
| **FID (Fréchet Inception Distance)** | 실제 이미지와 생성 이미지의 특징 분포 차이 (InceptionV3 활용) | `torch-fidelity` (`fid.compute_fid`) |
| **K‑NN based Precision / Recall** | Feature space(K‑NN)에서 실제와 생성 이미지간의 정밀도·재현율 측정 | `scikit-learn` KNN, 또는 `torch_fidelity` 의 `kernel_inception_distance` 옵션 |
| **Classifier Accuracy (조합)** | 사전학습된 (또는 직접 학습한) Fashion‑MNIST 분류기로 생성 이미지에 라벨을 예측 → **조건부 정확도** 측정 | 직접 `CNN` 학습 후 `accuracy` 계산 |

> **주의** – `torch-fidelity` 내부에서 `InceptionV3` 를 사용하므로, 28×28 이미지를 **Resize(299,299)** 로 자동 변환합니다. 이 과정이 오래 걸릴 수 있으니 GPU 사용을 권장합니다.

In [ ]:
# ============================================
# FID 계산
# ============================================
# 1) 실제 이미지 디렉터리와 생성 이미지 디렉터리를 준비
#    (예: real/와 fake/ 폴더에 10k PNG 저장)

# 실제 이미지 저장
real_dir = 'fid/real/'
fake_dir = 'fid/fake/'

os.makedirs(real_dir, exist_ok=True)
os.makedirs(fake_dir, exist_ok=True)

# 실제 이미지 (train_set) 저장
for idx, (img, _) in enumerate(train_set):
    if idx >= 10000: break
    torchvision.utils.save_image(
        img * 0.5 + 0.5,  # denorm
        os.path.join(real_dir, f'{idx:05d}.png')
    )

# 생성 이미지 저장 (클래스 별 1k씩)
for class_id in range(10):
    for i in range(1000):
        z = torch.randn(1, 100, device=device)
        label = torch.tensor([class_id], device=device)
        gen = generator(z, label).cpu()
        torchvision.utils.save_image(
            gen * 0.5 + 0.5,
            os.path.join(fake_dir, f'class{class_id}_{i:05d}.png')
        )

# 2) FID 계산 (torch-fidelity)
from torch_fidelity import calculate_metrics

metrics = calculate_metrics(
    input1=real_dir,
    input2=fake_dir,
    cuda=torch.cuda.is_available(),
    verbose=True,
    isc=False, # IS는 비활성화 (Inception 필요)
    fid=True,
    kid=False
)

print(f"FID: {metrics['frechet_inception_distance']:.4f}")

클래스‑조건부 정확도 (Conditional Consistency)

1. **분류기 사전 학습** – `torchvision.models.resnet18` 을 `FashionMNIST` 로 fine‑tune(1채널 → 3채널 변환 필요).
2. **생성 이미지에 라벨 예측** – `pred = classifier(gen_img)`.
3. **정확도** `accuracy = (pred.argmax(1) == label).float().mean()`.

> **Interpretation** – 10% 가 무작위 베이스라면, **50% 이상**이면 생성 이미지가 라벨 정보를 잘 반영하고 있다는 의미입니다. (예: 70%~80% 정도면 꽤 좋은 모델)

In [ ]:
# ============================================
# 클래스‑조건부 정확도 계산
# ============================================
# 예시: 간단한 CNN classifier
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1, 1), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, 1, 1), nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*7*7, 128), nn.ReLU(),
            nn.Linear(128, 10)
        )
    def forward(self, x):
        return self.classifier(self.features(x))

# 학습 후 저장된 모델 로드
classifier = SimpleCNN().to(device)
classifier.load_state_dict(torch.load('classifier.pth'))

# 조건부 정확도 측정
def conditional_accuracy(generator, classifier, n_samples=2000):
    generator.eval()
    classifier.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for _ in range(n_samples // 100):
            batch = 100
            z = torch.randn(batch, 100, device=device)
            labels = torch.randint(0, 10, (batch,), device=device)
            gen_imgs = generator(z, labels)
            logits = classifier(gen_imgs)
            preds = logits.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += batch
    return correct / total

print(f"Conditional classification accuracy: {conditional_accuracy(generator, classifier)*100:.2f}%")

### 모델 저장·로드 & 인퍼런스 함수

In [ ]:
# ============================================
# 모델 저장
# ============================================
torch.save(generator.state_dict(), 'generator.pth')
torch.save(discriminator.state_dict(), 'discriminator.pth')

# 로드 함수
def load_generator(checkpoint_path='generator.pth', device='cpu'):
    model = Generator().to(device)
    model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    model.eval()
    return model

# 인퍼런스 함수
def generate(class_id, n_samples=16, seed=None, device='cpu'):
    if seed is not None:
        torch.manual_seed(seed)
    gen = load_generator(device=device)
    z = torch.randn(n_samples, 100, device=device)
    labels = torch.full((n_samples,), class_id, dtype=torch.long, device=device)
    with torch.no_grad():
        imgs = gen(z, labels)
    # denorm
    imgs = imgs * 0.5 + 0.5
    return imgs.cpu()

# 예시 사용
samples = generate(class_id=7, n_samples=9, seed=42)  # 스니커즈
grid = torchvision.utils.make_grid(samples, nrow=3)
plt.figure(figsize=(4,4))
plt.title("Generated Sneakers")
plt.imshow(grid.permute(1,2,0))
plt.axis('off')
plt.show()